# 📊 Telecom X - Parte 2: Modelagem Preditiva do Churn

Neste notebook, damos continuidade ao projeto iniciado na Parte 1 (ETL e Análise Exploratória).

O objetivo aqui é **construir modelos preditivos capazes de prever o churn de clientes**, identificando os principais fatores que influenciam a evasão.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score

## 1. Carregar dados tratados

In [ ]:
# Carregar os dados tratados da Parte 1
df = pd.read_csv('dados_tratados.csv')
df.head()

## 2. Separar variáveis preditoras (X) e alvo (y)

In [ ]:
X = df.drop(columns=['Churn'])
y = df['Churn']

## 3. Pré-processamento (Encoding + Normalização)

In [ ]:
# Identificar colunas numéricas e categóricas
num_cols = X.select_dtypes(include=['int64','float64']).columns
cat_cols = X.select_dtypes(include=['object','category']).columns

# Pré-processador
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), num_cols),
        ('cat', OneHotEncoder(handle_unknown='ignore'), cat_cols)
    ])

# Separar treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

## 4. Treinar modelos preditivos (Regressão Logística e Random Forest)

In [ ]:
# Regressão Logística
log_reg = Pipeline(steps=[('preprocessor', preprocessor),
                          ('model', LogisticRegression(max_iter=1000))])
log_reg.fit(X_train, y_train)

# Random Forest
rf = Pipeline(steps=[('preprocessor', preprocessor),
                     ('model', RandomForestClassifier(n_estimators=200, random_state=42))])
rf.fit(X_train, y_train)

## 5. Avaliação dos modelos

In [ ]:
# Avaliar Regressão Logística
y_pred_log = log_reg.predict(X_test)
print('=== Regressão Logística ===')
print(classification_report(y_test, y_pred_log))
print('AUC:', roc_auc_score(y_test, log_reg.predict_proba(X_test)[:,1]))

# Avaliar Random Forest
y_pred_rf = rf.predict(X_test)
print('
=== Random Forest ===')
print(classification_report(y_test, y_pred_rf))
print('AUC:', roc_auc_score(y_test, rf.predict_proba(X_test)[:,1]))

## 6. Importância das variáveis (Random Forest)

In [ ]:
feature_names = (num_cols.tolist() + list(rf.named_steps['preprocessor']
                                          .transformers_[1][1]
                                          .get_feature_names_out(cat_cols)))
importances = rf.named_steps['model'].feature_importances_
indices = np.argsort(importances)[::-1]

plt.figure(figsize=(8,6))
plt.title('Importância das variáveis (Random Forest)')
plt.barh(range(10), importances[indices][:10], align='center')
plt.yticks(range(10), [feature_names[i] for i in indices[:10]])
plt.gca().invert_yaxis()
plt.show()

## 7. Conclusões Estratégicas

- Clientes com **contratos mensais** e **menor tempo de permanência (tenure baixo)** são mais propensos a cancelar.
- Preços altos de mensalidade também aparecem como fator importante.
- Sugestão: focar em programas de fidelização, descontos para novos clientes e pacotes de longo prazo.